# LGBM - Training

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import pickle as pkl
import wandb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.criterion.metrics import log_metrics
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

In [4]:
seed_everything()

### Setting Up WandB

In [5]:
WANDB_PROJECT = "Kaggle_ASE_2.0"
WANDB_NAME = f"LGBM-ASE-5"

In [6]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    save_code=True,
    job_type="train",
    config=config,
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


## Model Training

### Dataset Creation

In [7]:
df = pd.read_csv(Paths.FEATURE_ENGG_CSV_PATH)
df.shape

(17307, 227)

In [8]:
ids = df.loc[:, "essay_id"]
X, y = df.drop(columns=["score", "essay_id"]), df.loc[:, "score"]
del df

X.shape, y.shape

((17307, 225), (17307,))

### Callbacks

In [9]:
callbacks = [
    log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Constants

In [10]:
a = config.lgbm_a
b = config.lgbm_b

### Training Loop

In [11]:
def train_loop(X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=0.01,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=config.random_seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

In [12]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds = predictor.predict(X_valid) + a
    preds = preds.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
        }
    )
    log_metrics(oof, f"Fold {idx}")

    return oof

### Fold Loop

In [13]:
skf = StratifiedKFold(
    n_splits=config.lgbm_n_folds,
    random_state=config.random_seed,
    shuffle=True,
)

In [14]:
oof_df = pd.DataFrame()

for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
    id_valid = ids.loc[valid_idx].to_numpy()

    model, predictor = train_loop(X_train, y_train, X_valid, y_valid)

    with open(f"output/LGBM/{idx}.pkl", "wb") as file:
        pkl.dump({"predictor": predictor, "model": model}, file)

    oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
    oof_df = pd.concat([oof_df, oof])

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.44333	valid's QWK: 0.479699
[50]	train's QWK: 0.738785	valid's QWK: 0.760142
[75]	train's QWK: 0.800421	valid's QWK: 0.808415
[100]	train's QWK: 0.834861	valid's QWK: 0.844335
[125]	train's QWK: 0.857947	valid's QWK: 0.867896
[150]	train's QWK: 0.868671	valid's QWK: 0.876124
[175]	train's QWK: 0.873605	valid's QWK: 0.879976
[200]	train's QWK: 0.876336	valid's QWK: 0.884111
[225]	train's QWK: 0.879833	valid's QWK: 0.885599
[250]	train's QWK: 0.881905	valid's QWK: 0.891194
[275]	train's QWK: 0.882888	valid's QWK: 0.892075
[300]	train's QWK: 0.883589	valid's QWK: 0.893984
[325]	train's QWK: 0.884275	valid's QWK: 0.895287
[350]	train's QWK: 0.884951	valid's QWK: 0.895394
[375]	train's QWK: 0.885683	valid's QWK: 0.894936
[400]	train's QWK: 0.886069	valid's QWK: 0.894922
[425]	train's QWK: 0.886455	valid's QWK: 0.895791
[450]	train's QWK: 0.886732	valid's Q

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.469111	valid's QWK: 0.443412
[50]	train's QWK: 0.73839	valid's QWK: 0.72402
[75]	train's QWK: 0.794314	valid's QWK: 0.781048
[100]	train's QWK: 0.837703	valid's QWK: 0.825309
[125]	train's QWK: 0.859763	valid's QWK: 0.843257
[150]	train's QWK: 0.868086	valid's QWK: 0.852936
[175]	train's QWK: 0.872927	valid's QWK: 0.859017
[200]	train's QWK: 0.87591	valid's QWK: 0.859801
[225]	train's QWK: 0.878705	valid's QWK: 0.860792
[250]	train's QWK: 0.880296	valid's QWK: 0.864334
[275]	train's QWK: 0.881897	valid's QWK: 0.865291
[300]	train's QWK: 0.883218	valid's QWK: 0.86831
[325]	train's QWK: 0.884349	valid's QWK: 0.869801
[350]	train's QWK: 0.885126	valid's QWK: 0.869398
[375]	train's QWK: 0.88558	valid's QWK: 0.86842
Early stopping, best iteration is:
[301]	train's QWK: 0.883363	valid's QWK: 0.87003
Evaluated only: QWK
[LightGBM] [Info] Using self-defined o

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.454242	valid's QWK: 0.448251
[50]	train's QWK: 0.735971	valid's QWK: 0.726406
[75]	train's QWK: 0.799402	valid's QWK: 0.796604
[100]	train's QWK: 0.840819	valid's QWK: 0.844221
[125]	train's QWK: 0.858957	valid's QWK: 0.866715
[150]	train's QWK: 0.867708	valid's QWK: 0.869663
[175]	train's QWK: 0.872182	valid's QWK: 0.870304
[200]	train's QWK: 0.875685	valid's QWK: 0.876495
[225]	train's QWK: 0.878181	valid's QWK: 0.878343
[250]	train's QWK: 0.880851	valid's QWK: 0.881417
[275]	train's QWK: 0.882048	valid's QWK: 0.883741
[300]	train's QWK: 0.883564	valid's QWK: 0.881814
[325]	train's QWK: 0.884324	valid's QWK: 0.881224
Early stopping, best iteration is:
[270]	train's QWK: 0.881803	valid's QWK: 0.883741
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's Q

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.45003	valid's QWK: 0.450837
[50]	train's QWK: 0.73629	valid's QWK: 0.733911
[75]	train's QWK: 0.800026	valid's QWK: 0.796135
[100]	train's QWK: 0.841694	valid's QWK: 0.837613
[125]	train's QWK: 0.862241	valid's QWK: 0.857114
[150]	train's QWK: 0.869863	valid's QWK: 0.866902
[175]	train's QWK: 0.874466	valid's QWK: 0.874273
[200]	train's QWK: 0.877135	valid's QWK: 0.874656
[225]	train's QWK: 0.87988	valid's QWK: 0.875106
[250]	train's QWK: 0.881654	valid's QWK: 0.875704
[275]	train's QWK: 0.882969	valid's QWK: 0.87874
[300]	train's QWK: 0.884253	valid's QWK: 0.879289
[325]	train's QWK: 0.885379	valid's QWK: 0.880816
[350]	train's QWK: 0.88583	valid's QWK: 0.879764
[375]	train's QWK: 0.886271	valid's QWK: 0.88009
Early stopping, best iteration is:
[315]	train's QWK: 0.885247	valid's QWK: 0.880816
Evaluated only: QWK
[LightGBM] [Info] Using self-defined 

In [15]:
score = get_score(oof_df["score"], oof_df["pred_score"])
print(f'Overall Score: {score:<.4f}')
wandb.log({"CV/qwk_score": score})

Overall Score: 0.8859


In [16]:
wandb.log({"oof_table": wandb.Table(dataframe=oof_df)})

In [17]:
log_metrics(
    pd.DataFrame(
        {
            "score": oof_df.score.to_numpy(),
            "pred_score": oof_df.pred_score.to_numpy(),
        }
    ),
    "Overall",
)

In [18]:
wandb.finish()

CV/qwk_score,▁
CV/qwk_score,0.88595
